#### Imports

In [794]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.models import load_model

In [795]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [796]:
final_dataset.head()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,1
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,2
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,3
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,4


In [797]:
# columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter', 'age', 'y']
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter','y']
n_features = 4

In [798]:
grouped = final_dataset.groupby('product_id')
display(grouped)

In [799]:
scalers = {col: {} for col in columns}
scaled_data_list = []

for product_id, group in grouped:
    scaled_group = group.copy()
    for col in columns:
        scaler = StandardScaler()
        scaled_group[col] = scaler.fit_transform(group[[col]])
        scalers[col][product_id] = scaler
    scaled_data_list.append(scaled_group)

# Combinar todos los datos escalados en un solo DataFrame
scaled_data = pd.concat(scaled_data_list)

In [800]:
display(scaled_data[columns])

,plan_precios_cuidados,cust_request_qty,cust_request_tn,close_quarter,y
0,0.0,0.461891,-1.704215,-0.707107,-1.576908
1,0.0,-0.046911,-2.039835,-0.707107,-2.042103
2,0.0,0.786658,-0.435961,1.414214,-0.323111
3,0.0,-1.703222,-1.074260,-0.707107,-1.117043
4,0.0,2.865166,0.273781,-0.707107,0.353285
...,...,...,...,...,...
21615,0.0,-0.652507,-0.792584,-0.816497,-0.792584
21616,0.0,-0.268679,-0.649760,1.224745,-0.649760
21617,0.0,0.115148,-0.595870,-0.816497,-0.595870
21618,0.0,-0.012794,-0.290890,-0.816497,-0.290890


#### Funcion para preparar los datos y crear el modelo

El objetivo es predecir 2 dias en el futuro, por lo que la idea es re-armar el dataset. Donde el valor de X sera el conjunto de datos hasta N-2 e Y va a ser N (siendo N la cantidad de ventas para ese producto)

In [801]:
def prepare_data(data,validation=False):
    X, y = [], []
    unique_product_ids = data['product_id'].unique()
    
    max_seq_length = 0
    
    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        max_seq_length = max(max_seq_length, len(product_values))
    
    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        pad_length = max_seq_length - len(product_values)
        
        # Aplicar padding con ceros a la izquierda para igualar la longitud
        padded_product_values = np.pad(product_values[:, :-1], ((pad_length, 0), (0, 0)), mode='constant')
        
        if validation:
            X.append(padded_product_values)
            y.append(product_id)
        else:
            # Todas las observaciones menos las últimas dos. Como despues necesito agregar esos 2 registros
            # para predecir el future, le agrego un pad left extra de 2 para no romper el shape de la red
            X.append(np.pad(padded_product_values, ((2, 0), (0, 0)), mode='constant')[:-2])
            y.append(product_values[-1, -1])      # Última observación
    
    if validation:
        return np.array(X), np.array(y), max_seq_length
    else:
        return np.array(X), np.array(y), max_seq_length


Definimos la funcion para crear el modelo LSTM, sobre este se ejecutara la optimizacion bayesiana

In [802]:
n_steps = 2  # número de pasos de tiempo
epochs = 100
batch_size = 32
predictions = []

In [803]:
data = scaled_data[columns].values
X, y, max_seq_length = prepare_data(scaled_data)

#### Revisamos que la funcion genere registros coherentes

In [804]:
# Muestro todos los registros del ultimo producto
unique_product_ids = scaled_data['product_id'].unique()
product_data = scaled_data[scaled_data['product_id'] == unique_product_ids[0]] #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft
product_values = product_data[columns].values
display(product_values)

array([[ 0.        ,  0.46189074, -1.70421544, -0.70710678, -1.57690786],
       [ 0.        , -0.04691078, -2.0398351 , -0.70710678, -2.04210334],
       [ 0.        ,  0.78665767, -0.43596114,  1.41421356, -0.32311076],
       [ 0.        , -1.7032221 , -1.07425999, -0.70710678, -1.11704256],
       [ 0.        ,  2.865166  ,  0.27378143, -0.70710678,  0.35328467],
       [ 0.        ,  1.44701708,  0.35490544,  1.41421356,  0.4140519 ],
       [ 0.        , -0.59901455, -1.22422686, -0.70710678, -1.25068433],
       [ 0.        ,  2.23728327, -0.5685186 , -0.70710678, -0.44544444],
       [ 0.        , -0.59901455, -0.35136974,  1.41421356, -0.27688808],
       [ 0.        , -1.76817549, -0.07823625, -0.70710678,  0.14086658],
       [ 0.        ,  0.89491331,  0.39077184, -0.70710678,  0.6195406 ],
       [ 0.        , -0.01443409, -1.18431507,  1.41421356, -1.18702359],
       [ 0.        ,  0.00721704, -0.67712904, -0.70710678, -0.77989182],
       [ 0.        , -1.45423413, -1.0

In [805]:
display(X[0]) #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.46189074, -1.70421544, -0.70710678],
       [ 0.        , -0.04691078, -2.0398351 , -0.70710678],
       [ 0.        ,  0.78665767, -0.43596114,  1.41421356],
       [ 0.        , -1.7032221 , -1.07425999, -0.70710678],
       [ 0.        ,  2.865166  ,  0.27378143, -0.70710678],
       [ 0.        ,  1.44701708,  0.35490544,  1.41421356],
       [ 0.        , -0.59901455, -1.22422686, -0.70710678],
       [ 0.        ,  2.23728327, -0.5685186 , -0.70710678],
       [ 0.        , -0.59901455, -0.35136974,  1.41421356],
       [ 0.        , -1.76817549, -0.07823625, -0.70710678],
       [ 0.        ,  0.89491331,  0.39077184, -0.70710678],
       [ 0.        , -0.01443409, -1.18431507,  1.41421356],
       [ 0.        ,  0.00721704, -0.67712904, -0.70710678],
       [ 0.        , -1.45423413, -1.01800077, -0.70710678],
       [ 0.        ,  1.

In [806]:
display(y[0]) #<= cambiar el indice entre 0 y -1

0.3617453793178112

In [807]:
display(len(X[0]))

36

#### Entrenamiento del modelo

In [808]:
display(max_seq_length)

36

In [809]:
from keras.metrics import MeanSquaredError
from keras.regularizers import l2

def create_model(units, learning_rate, optimizer='adam', dropout=0.1, activation="tanh", depth=2, l2_penalty=0.001):
    display(f"Creating model with parameters: units={units}, learning_rate={learning_rate}, "
                f"optimizer={optimizer}, dropout={dropout}, activation={activation}, "
                f"depth={depth}, l2_penalty={l2_penalty}")
    
    model = Sequential()
    
    for i in range(depth):
        if i == 0:
            model.add(LSTM(units=units, return_sequences=True, input_shape=(max_seq_length, n_features), activation=activation))
        else:
            model.add(LSTM(units=units, return_sequences=True, activation=activation))
            model.add(Dropout(dropout))
    
    model.add(LSTM(units=units, return_sequences=False, activation=activation))
    model.add(Dropout(dropout))
    
    model.add(Dense(1))
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    
    # Usamos el error cuadratico medio, porque penaliza errores grandes. Como tenemos ciertos
    # productos "estrella", para los cuales hay mas ventas, y tener un error en lo mismos
    # puede implicar una gran diferencia de las estimaciones, decidimos probar con esta funcion de perdida.
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[MeanSquaredError()])
    return model

#### Armado de los modelos

In [810]:
from skopt.space import Real, Integer, Categorical


param_space = {
    'depth': [1, 2, 3],#, 4, 5, 6],
    'activation': ['relu', 'tanh', 'swish'], #'selu'
    'units': [64, 128], #256], #512
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': Categorical(categories=(0.001, 0.01), prior=None),#0.0001
    'epochs': [10, 20, 30, 50, 100],
    'batch_size': [128],#[32, 64, 128],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    # 'l2_penalty': [0.001, 0.01, 0.1]
}

In [811]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=5) mean_squared_error
early_stopping = EarlyStopping(monitor='mean_squared_error', patience=5)
callbacks = [early_stopping]

model = KerasRegressor(
    build_fn=create_model,
    verbose=1,
    units=64,
    learning_rate=0.01,
    dropout=0.1,
    activation="tanh",
    depth=1,
    l2_penalty=0.001,
    callbacks = callbacks
)

# Como ahora cada observacion, representa la serie de un producto, no hace falta usar 
# un cv especifico de time series, con el cv normal deberia ser suficiente.
# cv = TimeSeriesSplit(n_splits=5).split(X)

bayes_search = BayesSearchCV(
    model,
    param_space,
    # n_iter=50,
    # cv=5,
    verbose=1,
    # n_jobs=-1,
    scoring='neg_mean_squared_error',
)

/Users/mmarchetta/Desktop/LaboratorioIII/myenv/lib/python3.8/site-packages/skopt/space/space.py:116: UserWarning: Dimension [64, 128] was inferred to Integer(low=64, high=128, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(64, 128), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(


In [812]:
bayes_search.fit(X, y, verbose=1)

5/5 [==============================] - 2s 336ms/step - loss: 0.5325 - mean_squared_error: 0.5325
Epoch 20/50
5/5 [==============================] - 2s 311ms/step - loss: 0.5173 - mean_squared_error: 0.5173
Epoch 21/50
5/5 [==============================] - 2s 324ms/step - loss: 0.5315 - mean_squared_error: 0.5315
Epoch 22/50
5/5 [==============================] - 2s 396ms/step - loss: 0.4691 - mean_squared_error: 0.4691
Epoch 23/50
5/5 [==============================] - 2s 347ms/step - loss: 0.4346 - mean_squared_error: 0.4346
Epoch 24/50
5/5 [==============================] - 2s 373ms/step - loss: 0.4122 - mean_squared_error: 0.4122
Epoch 25/50
5/5 [==============================] - 2s 407ms/step - loss: 0.4071 - mean_squared_error: 0.4071
Epoch 26/50
5/5 [==============================] - 2s 382ms/step - loss: 0.4032 - mean_squared_error: 0.4032
Epoch 27/50
5/5 [==============================] - 2s 373ms/step - loss: 0.3682 - mean_squared_error: 0.3682
Epoch 28/50
5/5 [==============

/Users/mmarchetta/Desktop/LaboratorioIII/myenv/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


'Creating model with parameters: units=90, learning_rate=0.001, optimizer=adam, dropout=0.3, activation=relu, depth=1, l2_penalty=0.001'

Epoch 1/30
5/5 [==============================] - 7s 161ms/step - loss: 0.9486 - mean_squared_error: 0.9486
Epoch 2/30
5/5 [==============================] - 1s 136ms/step - loss: 0.7157 - mean_squared_error: 0.7157
Epoch 3/30
5/5 [==============================] - 1s 131ms/step - loss: 0.6659 - mean_squared_error: 0.6659
Epoch 4/30
5/5 [==============================] - 1s 139ms/step - loss: 0.6604 - mean_squared_error: 0.6604
Epoch 5/30
5/5 [==============================] - 1s 118ms/step - loss: 0.6417 - mean_squared_error: 0.6417
Epoch 6/30
5/5 [==============================] - 1s 124ms/step - loss: 0.6388 - mean_squared_error: 0.6388
Epoch 7/30
5/5 [==============================] - 1s 126ms/step - loss: 0.6171 - mean_squared_error: 0.6171
Epoch 8/30
5/5 [==============================] - 1s 133ms/step - loss: 0.6122 - mean_squared_error: 0.6122
Epoch 9/30
5/5 [==============================] - 1s 130ms/step - loss: 0.5980 - mean_squared_error: 0.5980
Epoch 10/30
5/5 [===========

/Users/mmarchetta/Desktop/LaboratorioIII/myenv/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/100
5/5 [==============================] - 12s 177ms/step - loss: 1.0773 - mean_squared_error: 1.0773
Epoch 2/100
5/5 [==============================] - 1s 193ms/step - loss: 0.7003 - mean_squared_error: 0.7003
Epoch 3/100
5/5 [==============================] - 1s 208ms/step - loss: 0.6482 - mean_squared_error: 0.6482
Epoch 4/100
5/5 [==============================] - 1s 210ms/step - loss: 0.5578 - mean_squared_error: 0.5578
Epoch 5/100
5/5 [==============================] - 1s 167ms/step - loss: 0.5487 - mean_squared_error: 0.5487
Epoch 6/100
5/5 [==============================] - 1s 201ms/step - loss: 0.4881 - mean_squared_error: 0.4881
Epoch 7/100
5/5 [==============================] - 1s 194ms/step - loss: 0.4635 - mean_squared_error: 0.4635
Epoch 8/100
5/5 [==============================] - 1s 215ms/step - loss: 0.4658 - mean_squared_error: 0.4658
Epoch 9/100
5/5 [==============================] - 1s 229ms/step - loss: 0.4402 - mean_squared_error: 0.4402
Epoch 10/100
5/5 [

/Users/mmarchetta/Desktop/LaboratorioIII/myenv/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


'Creating model with parameters: units=108, learning_rate=0.001, optimizer=adam, dropout=0.3, activation=tanh, depth=1, l2_penalty=0.001'

Epoch 1/100
7/7 [==============================] - 6s 99ms/step - loss: 0.9043 - mean_squared_error: 0.9043
Epoch 2/100
7/7 [==============================] - 1s 121ms/step - loss: 0.6454 - mean_squared_error: 0.6454
Epoch 3/100
7/7 [==============================] - 1s 118ms/step - loss: 0.5420 - mean_squared_error: 0.5420
Epoch 4/100
7/7 [==============================] - 1s 78ms/step - loss: 0.5134 - mean_squared_error: 0.5134
Epoch 5/100
7/7 [==============================] - 1s 121ms/step - loss: 0.4579 - mean_squared_error: 0.4579
Epoch 6/100
7/7 [==============================] - 1s 106ms/step - loss: 0.4343 - mean_squared_error: 0.4343
Epoch 7/100
7/7 [==============================] - 1s 114ms/step - loss: 0.4336 - mean_squared_error: 0.4336
Epoch 8/100
7/7 [==============================] - 1s 98ms/step - loss: 0.4193 - mean_squared_error: 0.4193
Epoch 9/100
7/7 [==============================] - 1s 117ms/step - loss: 0.4006 - mean_squared_error: 0.4006
Epoch 10/100
7/7 [====

BayesSearchCV(estimator=KerasRegressor(activation='tanh', build_fn=<function create_model at 0x7f9930f46b80>, callbacks=[<keras.src.callbacks.EarlyStopping object at 0x7f991272d730>], depth=1, dropout=0.1, l2_penalty=0.001, learning_rate=0.01, units=64),
              scoring='neg_mean_squared_error',
              search_spaces={'activation': ['relu', 'tanh', 'swish', 'selu'],
                             'batch_size': [128], 'depth': [1, 2, 3],
                             'dropout': [0.1, 0.2, 0.3],
                             'epochs': [10, 20, 30, 50, 100],
                             'learning_rate': Categorical(categories=(0.001, 0.01), prior=None),
                             'optimizer': ['adam', 'rmsprop', 'sgd'],
                             'units': [64, 128]},
              verbose=1)

#### Guardo los mejores hiperparametros

In [813]:
import json

# Obtener los mejores hiperparámetros
best_params = bayes_search.best_params_

# Guardar los mejores hiperparámetros en un archivo JSON
with open('best_params.json', 'w') as file:
    json.dump(best_params, file)

print('Los mejores hiperparámetros han sido guardados en best_params.json.')


Los mejores hiperparámetros han sido guardados en best_params.json.


In [815]:
with open('best_params.json', 'r') as file:
    best_params = json.load(file)

# Reconstruir el modelo con los mejores hiperparámetros
mejor_modelo = create_model(units=best_params['units'], learning_rate=best_params['learning_rate'], optimizer=best_params['optimizer'], dropout=best_params['dropout'], activation=best_params['activation'], depth=best_params['depth'])
mejor_modelo.fit(X, y)
# Verificar la configuración del nuevo modelo
print("Configuración del nuevo modelo:")
print(mejor_modelo.summary())

'Creating model with parameters: units=108, learning_rate=0.001, optimizer=adam, dropout=0.3, activation=tanh, depth=1, l2_penalty=0.001'

25/25 [==============================] - 7s 34ms/step - loss: 0.7117 - mean_squared_error: 0.7117
Configuración del nuevo modelo:
Model: "sequential_319"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_944 (LSTM)             (None, 36, 108)           48816     
                                                                 
 lstm_945 (LSTM)             (None, 108)               93744     
                                                                 
 dropout_625 (Dropout)       (None, 108)               0         
                                                                 
 dense_319 (Dense)           (None, 1)                 109       
                                                                 
Total params: 142669 (557.30 KB)
Trainable params: 142669 (557.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


#### Hago las predicciones

In [ ]:
# best_params = BayesSearchCV.best_params_

# best_units = best_params['build_fn__units']
# best_dropout_rate = best_params['build_fn__dropout_rate']
# best_optimizer = best_params['build_fn__optimizer']

# best_model = create_model(units=best_units, dropout_rate=best_dropout_rate, optimizer=best_optimizer)
# best_model.fit(X, y)

In [816]:
X_f, product_ids, _ = prepare_data(scaled_data, validation=True)

In [817]:
scaled_predictions = mejor_modelo.predict(X_f)
display(scaled_predictions)

25/25 [==============================] - 1s 12ms/step


array([[-0.95330447],
       [-0.7653332 ],
       [-0.6108019 ],
       [-0.5785983 ],
       [-0.3204311 ],
       [-0.97415614],
       [-0.773747  ],
       [-0.91845965],
       [-0.38515878],
       [-0.3495432 ],
       [-0.32098585],
       [-0.9125735 ],
       [-0.5617287 ],
       [-0.77879596],
       [-0.81576055],
       [-0.9669008 ],
       [-0.8987611 ],
       [-0.9500809 ],
       [-0.76777005],
       [-0.4095762 ],
       [-0.5714995 ],
       [-0.10306953],
       [-0.60230976],
       [-0.6332318 ],
       [-0.74818885],
       [-0.9346338 ],
       [-0.80672365],
       [-0.78262514],
       [-0.8647619 ],
       [-0.7223747 ],
       [-0.7602315 ],
       [-0.22296242],
       [-0.73055744],
       [-0.6673908 ],
       [-1.0452187 ],
       [-0.6059968 ],
       [-0.51235235],
       [-0.69112504],
       [-0.23870692],
       [-0.8033563 ],
       [-0.58786726],
       [-1.1362509 ],
       [-0.36009318],
       [-0.68690544],
       [-0.981196  ],
       [-0

In [818]:
scaled_predictions_1d =  scaled_predictions.reshape(-1)
display(scaled_predictions_1d)

array([-0.95330447, -0.7653332 , -0.6108019 , -0.5785983 , -0.3204311 ,
       -0.97415614, -0.773747  , -0.91845965, -0.38515878, -0.3495432 ,
       -0.32098585, -0.9125735 , -0.5617287 , -0.77879596, -0.81576055,
       -0.9669008 , -0.8987611 , -0.9500809 , -0.76777005, -0.4095762 ,
       -0.5714995 , -0.10306953, -0.60230976, -0.6332318 , -0.74818885,
       -0.9346338 , -0.80672365, -0.78262514, -0.8647619 , -0.7223747 ,
       -0.7602315 , -0.22296242, -0.73055744, -0.6673908 , -1.0452187 ,
       -0.6059968 , -0.51235235, -0.69112504, -0.23870692, -0.8033563 ,
       -0.58786726, -1.1362509 , -0.36009318, -0.68690544, -0.981196  ,
       -0.75638014, -0.9470714 , -0.7402187 , -0.6322093 , -0.30728036,
       -0.7471217 , -0.89507324, -0.69777197, -0.58765674, -0.39259717,
       -0.32232305, -0.7226498 , -0.39162508, -0.63217396, -0.61160046,
       -0.74466366, -0.7316096 , -0.6674385 , -0.9748343 , -0.7992588 ,
       -1.0527016 , -0.629736  , -0.8736253 , -0.39082977, -0.69

In [819]:
predictions_df = pd.DataFrame({
    'product_id': product_ids,
    'predicted_y': scaled_predictions_1d
})

# Iterar sobre cada fila del DataFrame predictions_df
for index, row in predictions_df.iterrows():
    # Obtener el ID del producto y la predicción escalada para este producto
    product_id = row['product_id']
    scaled_prediction = row['predicted_y']
    
    # Obtener el escalador correspondiente a 'predicted_y' para este producto
    scaler = scalers['y'][product_id]
    
    # Aplicar la inversa del escalador a la predicción 'predicted_y' para este producto
    inverse_scaled_prediction = scaler.inverse_transform([[scaled_prediction]])[0][0]
    
    # Reemplazar la predicción escalada con la predicción invertida en el DataFrame final
    predictions_df.at[index, 'predicted_y'] = inverse_scaled_prediction

# Guardar el DataFrame final con las predicciones invertidas
predictions_df.to_csv('predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.')


Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.
